### PySpark Project


In [0]:
#configuring spark
import findspark
findspark.init()
findspark.find()

In [0]:
from pyspark.sql import SparkSession
#starting application with two worker nodes
spark=SparkSession.builder.appName('FbPostAnalysis').master('local[2]').getOrCreate()
spark

In [0]:
df=spark\
    .read\
    .option('mode','FAILFAST')\
    .csv('../data/dataset_Facebook_cos.csv',header=True,inferSchema=True)
df.show()

In [0]:
df.printSchema()

In [0]:
# 1. The total number of posts made
df.count()

In [0]:
# 2. The percentage of the growth or decline of the page, in terms of likes
# (subscriptions on the page), from the first post to the latest post
import pyspark.sql.functions as f
first_post_likes=df.select('Page total likes').orderBy('Post Month','Post Weekday','Post Hour').first()['Page total likes']
last_post_likes=df.select('Page total likes').orderBy('Post Month','Post Weekday','Post Hour',ascending=False).first()['Page total likes']

# type(first_post_likes)
growth_percentage=f'{round((last_post_likes-first_post_likes)/last_post_likes*100)}%'
growth_percentage

In [0]:
# 3. Which month, on average, has the highest number of post interactions?
df.groupBy('Post Month').agg(f.avg('Total Interactions').alias('Average Monthly Interaction')).orderBy(f.desc('Average Monthly Interaction')).show(3)

In [0]:
# 5. Which hour of the day, on average, has the highest number of post interactions?
df.groupBy('Post Hour').agg(f.avg('Total Interactions').alias('Average Hourly Interaction')).orderBy(f.desc('Average Hourly Interaction')).show(3)

In [0]:
# 6. Determine if paid (promoted) posts have a higher correlation with a large number
# of post shares when compared to the post shares of organic (non-promoted) posts.
# This is to determine the commercial viability of investing in paid posts for
# promoting cosmetic products. Answer with either a Yes or a No, and provide the
# methodology of how you reached your conclusion
df.groupBy('Paid').agg(
    f.avg('Lifetime Post Total Reach').alias('Total Lifetime Reach'),
    f.avg('Lifetime Post Consumers').alias('Total Lifetime Consumers'),
    f.avg('Lifetime Engaged Users').alias('Total Lifetime Engaged'),
    f.avg('share').alias('share'),
).show()

# df.where(df['Paid']==1).corr('Paid','share')
'''
Ans: Yes, Promoted posts have more shares than non-promoted shares.
'''

In [0]:
# 7. Which post type (photo, video, status, or link) is the most attractive to people who
# have subscribed to your page (people who have liked the page)?
df.groupBy('Type')\
    .agg(
    f.sum('Lifetime Post reach by people who like your Page').alias('Number of people who liked page')
)\
    .orderBy(f.desc('Number of people who liked page'))\
    .show()

In [0]:
# 8. Which hour of the day is ideal for posting photographic content? Arrange the
# hours of the day according to the order of the Lifetime Post Impressions column?
df.groupBy('Post Hour')\
    .agg(
    f.avg('Lifetime Post Total Impressions').alias('Average Impressions')
)\
    .orderBy(f.desc('Average Impressions'))\
    .show()

In [0]:
# 9. Create an additional column with the name Likes-to-comment Ratio, with the
# column values having the equation: likes to comment ratio = like/comment
# Hint: Make sure the ratio is in a decimal format, and correct it to 2 decimal places

df.select("*",f.round(f.col('like')/f.col('comment'),2).alias('Likes-to-comment Ratio')).show()

In [0]:
# 10. Arrange post categories (1,2,3) in the descending order of the reach that they
# can accumulate on average
df.groupBy('Type').agg(
   f.ceil(f.avg('Lifetime Post Total Reach')).alias('Average Reach')
)\
    .orderBy(f.desc('Average Reach'))\
    .show()

In [0]:
# 11. Determine the standard deviation of the average post reach for each of the day
# hours. This is to determine if the time of the day is an ideal criterion to identify
# when to create posts
df.groupBy('Post hour').agg(
    f.stddev(f.col('Lifetime Post Total Reach'))
).show()

In [0]:
# 12. Is there any correlation between the number of post consumptions and the
# total interactions on the post?
df.corr('Lifetime Post Consumptions','Total Interactions')
'''
Ans: There is very weak relationship.
'''

In [0]:
# 13. Determine the two best days in a week to create posts, when people are
# extremely active on social media, based on the data that you have
# Hint: Question 13 can have a subjective answer. You are free to choose your own
# approach to determine the best days to post in a week. Make sure to validate your
# claims with the relevant code and explanation of your approach.
df.groupBy('Post Weekday').agg(
    f.count('Post Hour').alias('Average Hours'),
    f.avg('Total Interactions').alias('Average Interaction'),
    f.avg('comment').alias('Average Comment'),
    f.avg('like').alias('Average like'),
    f.avg('share').alias('Average share'),
).orderBy(f.desc('Average Hours'))\
    .show()
'''
Ans: According to results Monday and Thursday are two days best for social media posts interactions.
'''